In [1]:
CELL_TYPE = 'pDC'
N_GENES: int = 20
SEED = 'shap_studyID' #'disease_NOstudy' 'study_NOdisease' or 'int' or 'shap_studyID'
TEST_SPLIT_IDX: int = 1 #[0,4]

In [2]:
# Parameters
CELL_TYPE = "Plasma"
SEED = 5
TEST_SPLIT_IDX = 1


In [3]:
N_SPLITS: int = 5
N_TRIALS: int = 50

In [4]:
import os
import sys
from pyprojroot.here import here
import pandas as pd
import anndata as ad
import numpy as np
import math
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from sklearn.metrics import balanced_accuracy_score, f1_score
import optuna

import joblib
import pickle
import datetime

import collections

import xgboost
from sklearn.preprocessing import LabelEncoder

import scipy.sparse as ssp
import joblib

from dotenv import load_dotenv

In [5]:
load_dotenv()

True

# LOAD DATASET

In [6]:
train_adata = ad.read_h5ad(
    here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/data_cellTypes/EXTERNAL_{CELL_TYPE}.filtered.log1p.h5ad')
)

In [7]:
if SEED != 'all':
    gene_subset = np.load(here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/shap_gene_selection/gene_subsets_{N_GENES}/{CELL_TYPE}_{SEED}.npy'), allow_pickle=True)
    train_adata = train_adata[:,gene_subset]
    print(gene_subset)
elif SEED == 'all':
    print('Using all genes')
else:
    raise ValueError()

['ENSG00000117523' 'ENSG00000244687' 'ENSG00000155368' 'ENSG00000197956'
 'ENSG00000143110' 'ENSG00000112561' 'ENSG00000114423' 'ENSG00000178562'
 'ENSG00000254087' 'ENSG00000184557' 'ENSG00000079805' 'ENSG00000170476'
 'ENSG00000118260' 'ENSG00000115884' 'ENSG00000258315' 'ENSG00000115145'
 'ENSG00000231389' 'ENSG00000134470' 'ENSG00000009790' 'ENSG00000145220'
 'ENSG00000204843' 'ENSG00000170581' 'ENSG00000123268' 'ENSG00000141526'
 'ENSG00000107223' 'ENSG00000111796' 'ENSG00000136997' 'ENSG00000001167'
 'ENSG00000089327' 'ENSG00000239713' 'ENSG00000130724' 'ENSG00000106605'
 'ENSG00000152219' 'ENSG00000179218' 'ENSG00000169442' 'ENSG00000204257'
 'ENSG00000140379' 'ENSG00000127951' 'ENSG00000027697' 'ENSG00000030110'
 'ENSG00000170458' 'ENSG00000006125' 'ENSG00000133134' 'ENSG00000137193'
 'ENSG00000100911' 'ENSG00000111335' 'ENSG00000130830' 'ENSG00000100902'
 'ENSG00000104904' 'ENSG00000197965' 'ENSG00000107485' 'ENSG00000100365'
 'ENSG00000164483' 'ENSG00000137441' 'ENSG000001720

In [8]:
train_adata.shape

(542, 107)

In [9]:
train_adata.obs.disease.unique()

['RA', 'COVID', 'healthy', 'HIV', 'cirrhosis', 'CD', 'SLE', 'sepsis']
Categories (8, object): ['CD', 'COVID', 'HIV', 'RA', 'SLE', 'cirrhosis', 'healthy', 'sepsis']

In [10]:
train_adata.obs.sampleID.unique()

['SCGT00val_I0364_T0', 'SCGT00val_I036019_T0', 'SCGT00val_I036028_T0', 'SCGT00val_I036024_T0', 'SCGT00val_I036016_T0', ..., '10XGenomics_10XHC2_T0', '10XGenomics_10XHC3_T0', '10XGenomics_10XHC5_T0', '10XGenomics_10XHC7_T0', '10XGenomics_10XHC8_T0']
Length: 74
Categories (74, object): ['10XGenomics_10XHC1_T0', '10XGenomics_10XHC2_T0', '10XGenomics_10XHC3_T0', '10XGenomics_10XHC4_T0', ..., 'Savage2021_BRISL6_T0', 'Savage2021_BRISL7_T0', 'Savage2021_PIDA_T0', 'Savage2021_PIDB_T0']

In [11]:
all_idxs = np.arange(train_adata.obs.shape[0])
left_out_splits = [s[1] for s in StratifiedGroupKFold(n_splits=N_SPLITS).split(all_idxs, train_adata.obs.disease, train_adata.obs.sampleID)]

In [12]:
TRAIN_SPLIT_IDXS = [0,1,2,3,4]
VAL_SPLIT_IDX = (TEST_SPLIT_IDX + 1) % 5
TRAIN_SPLIT_IDXS.remove(TEST_SPLIT_IDX)
TRAIN_SPLIT_IDXS.remove(VAL_SPLIT_IDX)
TRAIN_SPLIT_IDXS, VAL_SPLIT_IDX, TEST_SPLIT_IDX

([0, 3, 4], 2, 1)

In [13]:
train_idxs = np.concatenate([left_out_splits[idx] for idx in TRAIN_SPLIT_IDXS])
val_idxs = left_out_splits[VAL_SPLIT_IDX]
test_idxs = left_out_splits[TEST_SPLIT_IDX]

### SUBSET DATASET INTO TRAIN/TEST/VAL SPLITS

In [14]:
X_train = train_adata.X[train_idxs]
X_test = train_adata.X[test_idxs]
X_val = train_adata.X[val_idxs]
X_train.shape, X_test.shape, X_val.shape

((333, 107), (107, 107), (102, 107))

In [15]:
y_train = train_adata.obs.iloc[train_idxs].disease.values.astype(str)
y_test = train_adata.obs.iloc[test_idxs].disease.values.astype(str)
y_val = train_adata.obs.iloc[val_idxs].disease.values.astype(str)
y_train.shape, y_test.shape, y_val.shape

((333,), (107,), (102,))

In [16]:
lenc = LabelEncoder()
y_train_enc = lenc.fit_transform(y_train)
y_val_enc = lenc.transform(y_val)
y_test_enc = lenc.transform(y_test)

### GENERATE F1 

In [17]:
def custom_f1_score(y_true, y_pred):
    return -f1_score(y_true, y_pred.argmax(1), average='weighted')

In [18]:
eval_metric=custom_f1_score
eval_metric_name='custom_f1_score'

def objective(trial):
    params = {
        'n_estimators': 1500,
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 250),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 5e-1, log=True),
    }
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, f'validation_0-{eval_metric_name}')
    es_callback = xgboost.callback.EarlyStopping(20, min_delta=0.001)
    xgb = xgboost.XGBClassifier(
        eval_metric=eval_metric,
        callbacks=[pruning_callback, es_callback],
        n_jobs=5,
        **params
    )
    xgb.fit(
        X_train, 
        y_train_enc, 
        verbose=0,
        eval_set=[(X_val, y_val_enc)],
    )
    trial.set_user_attr('best_iteration', xgb.best_iteration)

    return xgb.best_score

In [19]:
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=N_TRIALS, gc_after_trial=True)

[I 2025-05-15 18:15:31,147] A new study created in memory with name: no-name-68af6fb0-fad9-4243-b3cc-d0768ad5ecb4


[I 2025-05-15 18:15:31,223] Trial 0 finished with value: -0.03313 and parameters: {'max_depth': 9, 'min_child_weight': 238, 'subsample': 0.7587945476302645, 'colsample_bynode': 0.6387926357773329, 'learning_rate': 0.0026368755339723046}. Best is trial 0 with value: -0.03313.


[I 2025-05-15 18:15:31,552] Trial 1 finished with value: -0.333516 and parameters: {'max_depth': 5, 'min_child_weight': 15, 'subsample': 0.8795585311974417, 'colsample_bynode': 0.6410035105688879, 'learning_rate': 0.08148293210105287}. Best is trial 1 with value: -0.333516.


[I 2025-05-15 18:15:31,703] Trial 2 finished with value: -0.03313 and parameters: {'max_depth': 3, 'min_child_weight': 243, 'subsample': 0.8491983767203796, 'colsample_bynode': 0.29110519961044856, 'learning_rate': 0.003095566460242371}. Best is trial 1 with value: -0.333516.


[I 2025-05-15 18:15:31,863] Trial 3 finished with value: -0.03313 and parameters: {'max_depth': 6, 'min_child_weight': 77, 'subsample': 0.5722807884690141, 'colsample_bynode': 0.48875051677790415, 'learning_rate': 0.006109683510122491}. Best is trial 1 with value: -0.333516.


...

[I 2025-05-15 18:15:41,159] Trial 45 finished with value: -0.166667 and parameters: {'max_depth': 14, 'min_child_weight': 24, 'subsample': 0.44429429121234915, 'colsample_bynode': 0.4617605841893383, 'learning_rate': 0.012740049048980703}. Best is trial 41 with value: -0.363072.


[I 2025-05-15 18:15:41,304] Trial 46 pruned. Trial was pruned at iteration 0.


[I 2025-05-15 18:15:42,177] Trial 47 finished with value: -0.354172 and parameters: {'max_depth': 8, 'min_child_weight': 1, 'subsample': 0.3862494415863606, 'colsample_bynode': 0.5044857205780174, 'learning_rate': 0.07552190947318929}. Best is trial 41 with value: -0.363072.


[I 2025-05-15 18:15:42,312] Trial 48 pruned. Trial was pruned at iteration 0.


[I 2025-05-15 18:15:42,514] Trial 49 pruned. Trial was pruned at iteration 21.


In [20]:
out_dir = here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_{N_GENES}/study')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
joblib.dump(study,os.path.join(out_dir, f'{CELL_TYPE}_{SEED}_{TEST_SPLIT_IDX}_xgboost.pkl'))

['/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_20/study/Plasma_5_1_xgboost.pkl']

In [21]:
n_estimators = int(study.best_trial.user_attrs['best_iteration']*1.2)
xgb = xgboost.XGBClassifier(
        eval_metric=eval_metric,
        n_estimators=n_estimators,
        **study.best_trial.params
    )
xgb.fit(
    ssp.vstack((X_train, X_val)), 
    np.concatenate((y_train_enc, y_val_enc)),
    verbose=1,
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=0.5603530864218136,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False,
              eval_metric=<function custom_f1_score at 0x7fb6be8804a0>,
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.008819730257104119, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=0, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [22]:
out_dir = here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_{N_GENES}/best_model')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
joblib.dump(xgb, os.path.join(out_dir, f'{CELL_TYPE}_{SEED}_{TEST_SPLIT_IDX}_xgb.json'))

['/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_20/best_model/Plasma_5_1_xgb.json']

In [23]:
df_pred_test = pd.DataFrame(dict(
    cell_id=train_adata.obs.iloc[test_idxs].index.values,
    y_true=y_test, 
    y_true_code=y_test_enc, 
    y_pred=xgb.predict(X_test))).set_index('cell_id')

In [24]:
out_dir = here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_{N_GENES}/predictions')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
df_pred_test.to_csv(os.path.join(out_dir, f'{CELL_TYPE}_{SEED}_{TEST_SPLIT_IDX}_pred_test.zip'))

In [25]:
metrics_dict = dict(
    BAS=balanced_accuracy_score(y_true=df_pred_test.y_true_code, y_pred=df_pred_test.y_pred), WF1=f1_score(y_true=df_pred_test.y_true_code, y_pred=df_pred_test.y_pred,average='weighted'))

In [26]:
metrics_dict

{'BAS': 0.125, 'WF1': 0.019166798669412323}

In [27]:
metrics_df = pd.DataFrame.from_dict([metrics_dict]).assign(split_idx=TEST_SPLIT_IDX, gene_set_seed=SEED, cell_type=CELL_TYPE)
metrics_df

,BAS,WF1,split_idx,gene_set_seed,cell_type
0,0.125,0.019167,1,5,Plasma


In [28]:
out_dir = here(f'03_downstream_analysis/08_gene_importance/xgboost_external_validation/xgboost_TopN_genes/results_{N_GENES}/metrics')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
metrics_df.to_csv(os.path.join(out_dir, f'{CELL_TYPE}_{SEED}_{TEST_SPLIT_IDX}_metrics.zip'))